In [4]:
# Imports

import os
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from loguru import logger
from pathlib import Path

# Local imports
sys.path.append(r"./utils")
from utils import utils

In [5]:
# Constants

SAMPLE_DATA = False
DEBUG_MODEL = False

GROUP_NAME = "Country_gender_db_segment"  # TODO : Changed this for ethnicity segments. 
MODEL_DICT = \
            {"NN": {'epochs': 25} , 
             "LR": {'max_iter': 100}
            }
SELF_REPORTED_COLS = \
           ['age',
            # 'country_canada', 
            # 'country_united kingdom',
            # 'country_united states',
            # 'database_dating',
            # 'database_fb',
            # 'gender',
           ]
DATA_DROP_COLS = \
           ['Unnamed: 0', # index columns
            # 'pol',  # label column
            'gender', # self reported and filtered already
            # 'age', # self-reported  
            'country', # self reported and filtered already
            'userid', # index equivalent column 
            'pol_dat_us', # redundant columns with label
            'pol_dat_ca', # redundant columns with label
            'pol_dat_uk', # redundant columns with label
            'pol_fb_us', # redundant columns with label
            'database', # filtered already 
            # 'ethnicity.value' # filtered already
            ]
RESULTS_COLS = \
            ["Group Name", 
             "Model", 
             "Feature Set", 
             "Test AUC", 
             "Test Accuracy",
            ]
DATA_DIR = "./data/full/"
RESULTS_DIR = f"./results/full/{GROUP_NAME}/" #TODO: ensure that the folder exists
RESULTS_STATS_FILENAME = GROUP_NAME + '.csv'
RESULTS_MODEL_FILENAME_PREFIX = GROUP_NAME

if SAMPLE_DATA:
  DATA_DIR = "./data/sample/"
  RESULTS_DIR = f"./results/sample/{GROUP_NAME}/"
  ASSERT_DATA_SHAPE_0 = 31742
  ASSERT_DATA_SHAPE_1 = 2092
  DATA_DROP_COLS = DATA_DROP_COLS \
                  + ['Unnamed: 0.1'] #TODO: Regenerate sample with index=False and remove this

if DEBUG_MODEL:
  MODEL_DICT = \
            {"NN": {'epochs': 1} , 
             "LR": {'max_iter': 1}
            }

logger.debug(f"Started the script for {GROUP_NAME}.")

2021-08-18 17:32:26.639 | DEBUG    | __main__:<module>:60 - Started the script for Country_gender_db_segment.


In [6]:
# List of data file paths
results = []

folders = os.listdir(DATA_DIR)

for folder in tqdm(folders):
  
  if folder != "NO FILES":

    data = utils.get_segment_dataframe(data_dir = DATA_DIR, segment_to_run = folder)

    # Clean the data
    data = data.drop(DATA_DROP_COLS, axis=1)
    data = utils.get_clean_data(data)
    logger.debug(f"Data size after cleaning is {data.shape}")
    data_y = data['pol'].replace({"liberal": 1, "conservative": 0})
    data = data.drop('pol', axis = 1)
    all_features = data.columns

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(data, data_y, test_size = 0.2, random_state = 2021) 
    del data, data_y

    ## Define features for the various settings
    image_cols = list(map(str, range(1, 2049)))
    image_and_self_reported_cols = image_cols + SELF_REPORTED_COLS
    image_and_extracted_cols = [x for x in all_features if x not in SELF_REPORTED_COLS]
    image_and_self_reported_and_extracted_cols = all_features

    # which group it is being processed on 
    SEGMENT_NAME = folder
    logger.debug(f"Started the script for {SEGMENT_NAME}.")

    data_dict = {
              "Image Features" : image_cols,
              "Image and Self Reported Features" : image_and_self_reported_cols,
              "Image and Extracted Features": image_and_extracted_cols,
              "Image, Self-reported and Extracted Features": image_and_self_reported_and_extracted_cols
              }

    # Fit models and log results
    for data_name, data_set_features in tqdm(data_dict.items()):
      for model_name, model_params in MODEL_DICT.items():
        try:
          save_model_filepath = Path(RESULTS_DIR \
                        + SEGMENT_NAME\
                        + "_" + model_name \
                        + "_" + data_name.replace(" ","_").replace(",","").replace("-","_") \
                        + '.mdl')

          if save_model_filepath.is_file() or save_model_filepath.is_dir():
            logger.debug(f"{SEGMENT_NAME}, {model_name}, {data_name}: model already exists.")
            model = utils.read_model(model_name, save_model_filepath)
            logger.debug(f"{SEGMENT_NAME}, {model_name}, {data_name}: model read from disk.")
            
          else:
            logger.debug(f"{SEGMENT_NAME}, {model_name}, {data_name}: model training started.")
            model = utils.fit_model(model_name,
                                  X_train[data_set_features],
                                  y_train,
                                  model_params = model_params)

            utils.save_model(model, model_name, save_model_filepath)
            logger.debug(f"{GROUP_NAME}, {model_name}, {data_name}: model training ended and model saved.")
          
          auc, acc = utils.get_metrics(model_name,
                                      model,
                                      X_test[data_set_features],
                                      y_test)
                                      
          results.append([GROUP_NAME, model_name, data_name, auc, acc])
          logger.debug(f"{GROUP_NAME}, {model_name}, {data_name}: model training ended. AUC: {auc}, accuracy: {acc}")
        
        except Exception as error:
          logger.exception(error)
          logger.error(f"{SEGMENT_NAME}, {model_name}, {data_name}: Error occured!")

  0%|          | 0/9 [00:00<?, ?it/s]2021-08-18 17:32:28.541 | DEBUG    | __main__:<module>:15 - Data size after cleaning is (4000, 2080)
2021-08-18 17:32:28.600 | DEBUG    | __main__:<module>:32 - Started the script for UK_0_dating.
2021-08-18 17:32:28.602 | DEBUG    | __main__:<module>:52 - UK_0_dating, NN, Image Features: model already exists.
2021-08-18 17:32:29.023 | DEBUG    | __main__:<module>:54 - UK_0_dating, NN, Image Features: model read from disk.
2021-08-18 17:32:29.411 | DEBUG    | __main__:<module>:72 - Country_gender_db_segment, NN, Image Features: model training ended. AUC: 49.23, accuracy: 58.75
2021-08-18 17:32:29.412 | DEBUG    | __main__:<module>:52 - UK_0_dating, LR, Image Features: model already exists.
2021-08-18 17:32:29.415 | DEBUG    | __main__:<module>:54 - UK_0_dating, LR, Image Features: model read from disk.
2021-08-18 17:32:29.444 | DEBUG    | __main__:<module>:72 - Country_gender_db_segment, LR, Image Features: model training ended. AUC: 74.06, accuracy

In [7]:
# Save results summary to disk

save_results_filepath = Path(RESULTS_DIR + RESULTS_STATS_FILENAME)
utils.save_results(results_array = results, 
                     location = save_results_filepath,
                     columns = RESULTS_COLS)
print(pd.DataFrame(results,columns = RESULTS_COLS))
logger.debug(f"Script for {GROUP_NAME} finished.")

2021-08-18 17:33:06.896 | DEBUG    | utils.utils:save_results:263 - Results Saved.
2021-08-18 17:33:06.902 | DEBUG    | __main__:<module>:8 - Script for Country_gender_db_segment finished.


                   Group Name Model  \
0   Country_gender_db_segment    NN   
1   Country_gender_db_segment    LR   
2   Country_gender_db_segment    NN   
3   Country_gender_db_segment    LR   
4   Country_gender_db_segment    NN   
..                        ...   ...   
59  Country_gender_db_segment    LR   
60  Country_gender_db_segment    NN   
61  Country_gender_db_segment    LR   
62  Country_gender_db_segment    NN   
63  Country_gender_db_segment    LR   

                                    Feature Set  Test AUC  Test Accuracy  
0                                Image Features     49.23          58.75  
1                                Image Features     74.06          70.00  
2              Image and Self Reported Features     53.59          60.12  
3              Image and Self Reported Features     74.40          68.50  
4                  Image and Extracted Features     52.95          60.12  
..                                          ...       ...            ...  
59    